**Part 1**

In [1]:
# Load librariers
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#create pathway for savingg and uploading datasets
path = r'C:\Users\beatr\Documents\23-05-2023-Instacart Basket Analysis'

In [3]:
#import dataset
customers_df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [27]:
customers_df.shape

(206209, 10)

In [4]:
# check the import of data with column names
customers_df.head(10)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


4. **Rename & drop columns

In [5]:
#Rename columns
customers_df = customers_df.rename(columns={'First Name':'first_name', 'Surnam':'last_name', 'Gender':'sex', 'Age':'age'})


2. **Discriptive Statistics**
check basic stats of database

In [6]:
# Use decripe to identify basic statistical information
customers_df.describe()

,user_id,age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [7]:
#Check data types
customers_df.dtypes

user_id          int64
first_name      object
last_name       object
sex             object
STATE           object
age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [8]:
#Change user_id to a integer
customers_df['user_id'] = customers_df['user_id'].astype('int64')

5. **check for mixed data types**

In [9]:
#check for mixed data types
for col in customers_df.columns.tolist():
  weird = (customers_df[[col]].applymap(type) != customers_df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers_df[weird]) > 0:
    print (col)
#we identify that first_name is mixed 

first_name


In [10]:
#Fix mixed data type by converting to a str
customers_df['first_name'] = customers_df['first_name'].astype('str')

In [11]:
#rerun to check mixed data
for col in customers_df.columns.tolist():
  weird = (customers_df[[col]].applymap(type) != customers_df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers_df[weird]) > 0:
    print (col)

5. **Missing Values**

In [12]:
#check for null values
customers_df.isnull().sum()
#All good we have no missing values

user_id         0
first_name      0
last_name       0
sex             0
STATE           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

5. **Duplicate values**

In [13]:
# Check for duplicates 
df_dups = customers_df[customers_df.duplicated()]
df_dups
# we have no duplicates

,user_id,first_name,last_name,sex,STATE,age,date_joined,n_dependants,fam_status,income


6. Combine your customer data with the rest of your prepared Instacart data

In [14]:
# step 1 import the complete orders_products_merged dataset
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_agg_grouped.pkl'))

In [15]:
#check import
ords_prods_merge.head(20)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_spend,spender_type,median_orders,customer_frequency
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,False,196,1,1,...,Mid-range product,Regularly busy,Least busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,False,196,1,1,...,Mid-range product,Regularly busy,Least busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,False,196,1,1,...,Mid-range product,Least busy,Least busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,False,196,1,1,...,Mid-range product,Least busy,Least busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
5,3367565,1,6,2,7,19.0,False,196,1,1,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
6,550135,1,7,1,9,20.0,False,196,1,1,...,Mid-range product,Regularly busy,Busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
7,3108588,1,8,1,14,14.0,False,196,2,1,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
8,2295261,1,9,1,16,0.0,False,196,4,1,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
9,2550362,1,10,4,8,30.0,False,196,1,1,...,Mid-range product,Least busy,Least busiest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [16]:
#Recheck data types of orders/products data set
ords_prods_merge.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
new_customer                  bool
product_id                   int64
add_to_cart_order            int64
reordered                    int64
Unnamed: 0_y                 int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
avg_spend                  float64
spender_type                object
median_orders              float64
customer_frequency          object
dtype: object

In [17]:
#delete merge flag for next merge
del ords_prods_merge['_merge']

In [18]:
#check shape of data
ords_prods_merge.shape

(32404859, 25)

In [19]:
customers_df.shape

(206209, 10)

In [20]:
#merge data sets on user_id
df_merge = customers_df.merge(ords_prods_merge, on = 'user_id', indicator = True)

In [21]:
df_merge.shape

(32404859, 35)

In [22]:
df_merge['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [23]:
df_merge.head(30)

,user_id,first_name,last_name,sex,STATE,age,date_joined,n_dependants,fam_status,income,...,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_spend,spender_type,median_orders,customer_frequency,_merge
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Busiest days,Average orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
1,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Regularly busy,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
2,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Busiest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
3,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Regularly busy,Average orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
4,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Least busiest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
5,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Busiest days,Average orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
6,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Busiest days,Average orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
7,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Regularly busy,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
8,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Busiest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
9,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Regularly busy,Average orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both


In [24]:
# check the outer join to see how many values dropped
df_merge_outer = customers_df.merge(ords_prods_merge, on = 'user_id', how = 'outer', indicator = True)

In [25]:
df_merge_outer['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [26]:
# export merged product order data set as pickle
#df_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'instacard_basket_merged.pkl'))